In [11]:
!pip install scikit-learn numpy scipy



ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [13]:
import nltk



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\darkk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
import pandas as pd

In [ ]:
pp

In [17]:
df = pd.read_csv("spotify_millsongdata.csv")

In [19]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [21]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [23]:
df.shape

(57650, 4)

In [25]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [27]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [31]:
df['text'][0]

'Sleeping late on  \r\nWarm afternoon  \r\nWaking up to  \r\nBright silver moon  \r\n  \r\nSilent words from  \r\nMy heart to you  \r\nMy empty filled  \r\nFilled up with you  \r\n  \r\nRockets fly  \r\nRockets fly  \r\nI slip another smile in your pocket  \r\nMy heart is racing to you like a rocket  \r\n  \r\nRockets fly  \r\nRockets fly  \r\nI slip another smile in your pocket  \r\nMy heart is racing to you like a rocket  \r\nThe two of us we dream like one  \r\nOur energy could light the sun  \r\nThe two of us take breath as one  \r\nThe two of us... The two of us...\r\n\r\n'

In [35]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [37]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [76]:
import nltk
from nltk.stem.porter import PorterStemmer  # Corrected import

stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)


In [82]:
tokenization("you are beautiful")  # Correct


'you are beauti'

In [104]:
 df['text'].apply(lambda x: tokenization(x))

0       sleep late on warm afternoon wake up to bright...
1       the song a robin sing through year of endless ...
2       i realiz , realiz i get a littl bit wild somet...
3       if you do n't open your eye then you 're gon n...
4       `` a situat `` well i live here with you in , ...
                              ...                        
4995    in thi world of right and wrong the hardest pa...
4996    ( ver 1 ) so young in love , they could n't wa...
4997    without you stand by my side love and good for...
4998    i met you on a midway at a fair last year and ...
4999    whenev i touch you late , you seem to pull awa...
Name: text, Length: 5000, dtype: object

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("Imports successful!")


Imports successful!


In [112]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')


In [124]:
matrix=tfidvector.fit_transform(df['text'])

In [130]:
similarity=cosine_similarity(matrix)

In [132]:
similarity[0]

array([1.        , 0.01876388, 0.03414126, ..., 0.01975145, 0.06696221,
       0.01853728])

In [138]:
df[df['song'] == 'Good Old Days'].index[0]

4553

recommender function

In [148]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:5]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [150]:
recommendation("Good Old Days")

["The Goonies 'r' Good Enough",
 'Old Days',
 'You Give Good Love',
 'Good Day Sunshine']

In [152]:
import pickle
pickle.dump(similarity, open('similarity.pkl', 'wb'))
pickle.dump(df, open('df.pkl', 'wb'))